# 실습 5: Tools 생성

## 목표
- Disaster Tools 구현 (소방서 검색, 기상 정보, 위키피디아)
- Browser Tool 구현 (뉴스 검색)

## 5-1. 환경 변수 로드

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

REGION = os.getenv('AWS_REGION', 'us-west-2')
WINDY_API_KEY = os.getenv('WINDY_API_KEY', '')

print(f"🌍 Region: {REGION}")
print(f"🌤️ Windy API Key: {'설정됨' if WINDY_API_KEY else '미설정'}")

## 5-2. config.py 생성 (먼저)

In [ ]:
config_content = f'''"""Configuration settings"""

WINDY_API_KEY = "{WINDY_API_KEY}"
WINDY_API_URL = "https://api.windy.com/api/point-forecast/v2"
'''

os.makedirs('agent/deploy/runtime', exist_ok=True)
with open('agent/deploy/runtime/config.py', 'w', encoding='utf-8') as f:
    f.write(config_content)

print("✅ config.py 생성 완료")

## 5-3. 디렉토리 구조 생성

In [ ]:
import os

dirs = [
    'agent/deploy/runtime/tool_use',
    'agent/deploy/runtime/tool_use/browser_tool',
]

for dir_path in dirs:
    os.makedirs(dir_path, exist_ok=True)
    init_file = os.path.join(dir_path, '__init__.py')
    if not os.path.exists(init_file):
        open(init_file, 'w').close()

print("✅ 디렉토리 구조 생성 완료")

## 5-4. Disaster Tools 생성

In [ ]:
config_content = f'''"""Configuration settings"""

WINDY_API_KEY = "{WINDY_API_KEY}"
WINDY_API_URL = "https://api.windy.com/api/point-forecast/v2"
'''

with open('agent/deploy/runtime/config.py', 'w', encoding='utf-8') as f:
    f.write(config_content)

print("✅ config.py 생성 완료")

## 5-4. Disaster Tools 생성

In [ ]:
disaster_tools_content = f'''"""화재 대응 도구 모음"""

from typing import Dict, Any
import wikipedia as wiki
import requests
from strands import tool
import boto3
import json
import math
from config import WINDY_API_KEY, WINDY_API_URL

@tool
def wikipedia(query: str) -> Dict[str, Any]:
    """위키피디아에서 정보를 검색합니다."""
    try:
        wiki.set_lang("ko")
        page = wiki.page(query)
        summary = page.summary
        if len(summary) > 500:
            summary = summary[:500] + "..."
        return {{
            "success": True,
            "title": page.title,
            "summary": summary,
            "url": page.url
        }}
    except Exception as e:
        return {{"success": False, "error": str(e)}}

@tool
def find_fire_station(address: str) -> str:
    """화재 발생 시 주소를 기반으로 가까운 소방서 5곳을 벡터 검색으로 찾습니다."""
    try:
        bedrock = boto3.client("bedrock-runtime", region_name="{REGION}")
        s3vectors = boto3.client("s3vectors", region_name="{REGION}")
        
        # 주소를 임베딩으로 변환
        embedding_response = bedrock.invoke_model(
            modelId="amazon.titan-embed-text-v2:0",
            body=json.dumps({{
                "inputText": address,
                "dimensions": 1024,
                "normalize": True
            }})
        )
        
        query_embedding = json.loads(embedding_response["body"].read())["embedding"]
        
        # S3 Vectors로 유사도 검색
        search_response = s3vectors.query_vectors(
            vectorBucketName="firestation-location-xy",
            indexName="fire-station",
            queryVector={{"float32": query_embedding}},
            topK=5,
            returnMetadata=True
        )
        
        if search_response.get("vectors"):
            result = f"화재 발생 주소: {{address}}\\n\\n가까운 소방서 5곳:\\n\\n"
            for idx, item in enumerate(search_response["vectors"], 1):
                meta = item.get("metadata", {{}})
                result += f"{{idx}}. {{meta.get('name')}}\\n"
                result += f"   - 위도: {{meta.get('latitude')}}, 경도: {{meta.get('longitude')}}\\n"
                result += f"   - 주소: {{meta.get('address')}}\\n"
                result += f"   - 전화번호: {{meta.get('phone')}}\\n\\n"
            return result
        return "소방서를 찾을 수 없습니다."
    except Exception as e:
        return f"오류 발생: {{str(e)}}"

@tool
def get_weather_info(latitude: float, longitude: float) -> str:
    """Windy API를 사용해 기상 정보를 조회합니다."""
    try:
        payload = {{
            "lat": latitude,
            "lon": longitude,
            "model": "gfs",
            "parameters": ["wind", "temp", "rh", "pressure"],
            "levels": ["surface"],
            "key": WINDY_API_KEY
        }}
        
        response = requests.post(WINDY_API_URL, json=payload, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if data and data.get("ts"):
            idx = 0
            wind_u = data.get("wind_u-surface", [0])[idx]
            wind_v = data.get("wind_v-surface", [0])[idx]
            temp_k = data.get("temp-surface", [273.15])[idx]
            humidity = data.get("rh-surface", [0])[idx]
            pressure = data.get("pressure-surface", [1013])[idx]
            
            wind_speed = math.sqrt(wind_u**2 + wind_v**2)
            wind_dir = (math.atan2(wind_u, wind_v) * 180 / math.pi + 180) % 360
            temp_c = temp_k - 273.15
            
            result = f"위도: {{latitude}}, 경도: {{longitude}}\\n"
            result += f"온도: {{temp_c:.1f}}°C\\n"
            result += f"풍속: {{wind_speed:.1f}} m/s\\n"
            result += f"풍향: {{wind_dir:.0f}}°\\n"
            result += f"습도: {{humidity}}%\\n"
            result += f"기압: {{pressure}} hPa"
            return result
        return "기상 정보를 가져올 수 없습니다."
    except Exception as e:
        return f"기상 정보 조회 오류: {{str(e)}}"
'''

with open('agent/deploy/runtime/tool_use/disaster_tools.py', 'w', encoding='utf-8') as f:
    f.write(disaster_tools_content)

print("✅ disaster_tools.py 생성 완료")

## 5-5. Browser Tool 생성

In [ ]:
browser_tool_content = '''"""Browser tool using Playwright"""

from strands_tools import browser_tool

# Playwright 기반 브라우저 도구
# 뉴스 검색 및 웹 페이지 정보 수집에 사용
'''

with open('agent/deploy/runtime/tool_use/browser_tool/playwright.py', 'w', encoding='utf-8') as f:
    f.write(browser_tool_content)

print("✅ browser_tool/playwright.py 생성 완료")

## 5-6. Agent Factory에 Tools 추가

In [ ]:
# factory.py 업데이트
import re

with open('agent/deploy/runtime/agent/factory.py', 'r', encoding='utf-8') as f:
    factory_content = f.read()

# import 추가
if 'from tool_use.disaster_tools' not in factory_content:
    factory_content = factory_content.replace(
        'from memory.hooks import',
        '''from tool_use.disaster_tools import wikipedia, find_fire_station, get_weather_info
from strands_tools import browser_tool
from memory.hooks import'''
    )

# tools 추가
factory_content = factory_content.replace(
    'tools=[],  # lab5에서 추가',
    'tools=[browser_tool, wikipedia, find_fire_station, get_weather_info],'
)

with open('agent/deploy/runtime/agent/factory.py', 'w', encoding='utf-8') as f:
    f.write(factory_content)

print("✅ factory.py에 Tools 추가 완료")

## ✅ 실습 5 완료!

### 생성된 파일
- `agent/deploy/runtime/config.py`
- `agent/deploy/runtime/tool_use/disaster_tools.py`
- `agent/deploy/runtime/tool_use/browser_tool/playwright.py`

### 다음 단계
👉 **lab6_agent_runtime.ipynb**